In [ ]:
# Libraries

In [ ]:
import numpy as np

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
import seaborn as sns

In [ ]:
import cv2 as cv

In [ ]:
from sklearn import linear_model

In [ ]:
import warnings

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
plt.style.use('seaborn')

In [ ]:
# Helping Functions

In [ ]:
def plot_image(image):
    plt.imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
    plt.show(block=True)

In [ ]:
def display_image(image):
    cv.imshow('image', image)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [ ]:
def MinMaxScalar(x, a=-1, b=1):
    N = (x - x.min()) * (b - a)
    D = (x.max() - x.min())
    return (N / D) + a

In [ ]:
# Activate Functions

In [ ]:
def sigmoid(t):
    return 1 / (1 + np.exp(-t))

In [ ]:
def relu(t):
    return np.maximum(0, t)

In [ ]:
def swish(t):
    return t / (1 + np.exp(-t))

In [ ]:
def softplus(t):
    return np.log(1 + np.exp(t))

In [ ]:
# Loadings

In [ ]:
def preprocess_line(image):
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    blur = cv.bilateralFilter(gray, 3, 75, 75)
    thresh = cv.adaptiveThreshold(blur, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 11, 2)
    k = 5
    kernal = np.ones((k, k), np.uint8)
    dilate = cv.dilate(thresh, kernal, iterations=1)
    k = 21
    kernal = np.ones((k, k), np.uint8)
    erode = cv.erode(dilate, kernal, iterations=1)
    return erode

In [ ]:
line1 = cv.imread('line1.jpg')